<a target="_blank" href="https://colab.research.google.com/github/trainocate-japan/Machine-Learning-and-Deep-Learning-Hands-on/blob/main/exercise/6_ディープラーニング/6-1_TensorFlow_Kerasによる自動車の燃費予測.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 6-1_TensorFlow/Kerasによる自動車の燃費予測
このノートブックでは、TensorFlow / Kerasで回帰の予測モデルを作成します。<br>
予測を行うテーマは1970年代後半から1980年台初めの自動車の燃費を予測することです。

[Keras公式ドキュメント](https://keras.io/ja/)

## ライブラリのインポート

In [2]:
# データを処理するための基本的なライブラリ
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

# scikit-learnから必要なライブラリをインポート
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

!pip install tensorflow
# TensorFlow/Kerasで使用
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping # 早期終了


  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached wrapt-1.14.1-cp39-cp39-win_amd64.whl (35 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'error'


    ERROR: Command errored out with exit status 1:
     command: 'c:\users\fujitama\onedrive - トレノケート株式会社\デスクトップ\doc_mac0025g\mlho\_venv\scripts\python.exe' 'c:\users\fujitama\onedrive - トレノケート株式会社\デスクトップ\doc_mac0025g\mlho\_venv\lib\site-packages\pip\_vendor\pep517\_in_process.py' prepare_metadata_for_build_wheel 'C:\Users\fujitama\AppData\Local\Temp\tmp_zv_ati5'
         cwd: C:\Users\fujitama\AppData\Local\Temp\pip-install-z7d3q154\jax
    Complete output (13 lines):
    Error in sitecustomize; set PYTHONVERBOSE for traceback:
    SyntaxError: (unicode error) 'utf-8' codec can't decode byte 0x83 in position 0: invalid start byte (sitecustomize.py, line 7)
    running dist_info
    creating C:\Users\fujitama\AppData\Local\Temp\pip-modern-metadata-0h9fk6d7\jax.egg-info
    writing C:\Users\fujitama\AppData\Local\Temp\pip-modern-metadata-0h9fk6d7\jax.egg-info\PKG-INFO
    writing dependency_links to C:\Users\fujitama\AppData\Local\Temp\pip-modern-metadata-0h9fk6d7\jax.egg-info\dependenc

ModuleNotFoundError: No module named 'tensorflow'

Google Colaboratory上での出力のデフォルト設定

In [ ]:
# pandasのDataframeの出力
pd.set_option('display.max_columns', 500) # 表示列の最大
pd.set_option('display.max_rows', 500) # 表示行の最大
pd.set_option('display.unicode.east_asian_width', True) # 日本語出力時にヘッダのずれを解消
pd.options.display.float_format = '{:,.5f}'.format # 表示桁数の設定

# ノートブックの表示桁数設定。この設定はprint文には作用せず、セルの最後に書いたものを出力する際に適用されます。
%precision 3
# numpy配列の指数表示禁止設定
np.set_printoptions(suppress=True)
# numpy配列の表示桁数設定
np.set_printoptions(precision=3)

## データの準備
今回使用するデータはUCI Machine Learning Repositoryから公開されているAuto MPG データセットのコピーです。<br>
downloaded from : https://archive.ics.uci.edu/ml/datasets/auto+mpg

#### データを取り込む
- pandasのread_csvメソッドを使用して、mlho/data/auto_mpg.csvファイルを読み込みます
- 読み込んだものは変数df_auto_mpgに代入します

In [ ]:
# csvファイルを読み込みます
df_auto_mpg = pd.read_csv("/content/drive/MyDrive/mlho/data/auto_mpg.csv")

#### データを確認する
- MPG : 燃費（目的変数）
- Cylinders : シリンダーの数
- Displacement : 排気量
- Horsepower : 馬力
- Weight : 重量
- Acceleration : 加速度
- Model Year : モデル年
- USA, Europe, Japan ： 生産国のOne-Hot表現

In [ ]:
# 読み込んだデータを確認します
df_auto_mpg.head()

In [ ]:
# df_auto_mpgのデータ要約を確認
df_auto_mpg.info()

In [ ]:
# df_auto_mpgの統計情報を確認
df_auto_mpg.describe()

今回のデータセットには、Horsepowerに値の含まれていない「欠損値」が含まれており、そのままではニューラルネットワークで学習をすることができません。<br>
欠損値の補い方は、データによりいくつかの方法がありますが、今回は単純に削除します。

In [ ]:
# DataFrameのdropnaメソッドを使用して、欠損値の含まれる行を削除する
df_auto_mpg = df_auto_mpg.dropna()

In [ ]:
# 再度、df_auto_mpgのデータ要約を確認
df_auto_mpg.info()

#### 説明変数と目的変数を切り出す

In [ ]:
# 目的変数にするMPG以外をすべて説明変数にする
x = df_auto_mpg.drop(columns='MPG')
x.tail(4)

In [ ]:
# 目的変数はMPG
y = df_auto_mpg["MPG"]
y.tail(4)

#### データを訓練データと検証データに分割する

In [ ]:
# 訓練データと検証データに分割（80%を訓練用に使用）
train_x, val_x, train_y, val_y = train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=5) 

### データのスケールを揃える
回帰の場合はデータにより目的変数も標準化した方が良いケースがある為目的変数も標準化する。

説明変数を標準化

In [ ]:
# 訓練データ説明変数の各列の平均を計算する
train_x_mean = train_x.mean()
train_x_mean.head()

In [ ]:
# 訓練データ説明変数の各列の標準偏差を計算する
train_x_std = train_x.std()
train_x_std.head()

In [ ]:
# 訓練データ説明変数の標準化を行う
train_x_scaled = (train_x - train_x_mean) / train_x_std
train_x_scaled.head()

目的変数を標準化

In [ ]:
# 訓練データ目的変数の各列の平均を計算する
train_y_mean = train_y.mean()
train_y_mean

In [ ]:
# 訓練データ目的変数の各列の標準偏差を計算する
train_y_std = train_y.std()
train_y_std

In [ ]:
# 訓練データ目的変数の標準化を行う
train_y_scaled = (train_y - train_y_mean) / train_y_std
train_y_scaled.head()

検証データを標準化

In [ ]:
# 検証データ説明変数の標準化を行う
val_x_scaled = (val_x - train_x_mean) / train_x_std
val_x_scaled.head()

In [ ]:
# 検証データ目的変数の標準化を行う
val_y_scaled = (val_y - train_y_mean) / train_y_std
val_y_scaled.head()

## モデルの定義

In [ ]:
# ニューラルネットワークのモデルを定義する際に各パラメータの初期値が決定されます
# その初期値が毎回異ならないように乱数シードをこのタイミングで固定します
tf.random.set_seed(0)

# モデルオブジェクトを用意し必要な層を追加していく
model = Sequential()

# 中間層1層目
model.add(Dense(3, input_shape=(train_x.shape[1],)))
model.add(Activation('relu'))
# 中間層2層目
model.add(Dense(3))
model.add(Activation('relu'))
# 出力層
model.add(Dense(1)) # 回帰の場合は活性化関数なし（恒等関数）

# 最適化手法としてAdam、誤差関数として平均二乗誤差を設定
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')

model.summary()

## モデルの学習

In [ ]:
history = model.fit(train_x_scaled, train_y_scaled, batch_size=128, epochs=700, validation_data = (val_x_scaled, val_y_scaled), verbose=1)

## 評価

ニューラルネットワークの学習が順調に進んだかどうかを確認するには、エポックごとに誤差関数がどのように変化したかを確認することが有効です。

訓練データに対する誤差関数と検証データに対する誤差関数を並べて表示し、二つを見比べることで誤差が順調に減少しているか、過学習を起こしていないか考察することができます。

Kerasではfitメソッドの戻り値のhistoryオブジェクトに学習の履歴が格納されています。今回はhistoryオブジェクトのhistoryプロパティにlossとval_lossだけが格納されているので、それを可視化してみましょう。

panasのDataFrameにはmatplotlibをラッパーしたplotメソッドが用意されています。<br>
DataFrameのplotメソッドを使用することで、DataFrameの中身を簡単にグラフ化することができます。<br>
[DataFrameのplotメソッドAPIリファレンス](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.plot.html)

In [ ]:
df_history = pd.DataFrame(history.history)
df_history.plot(figsize=(10, 6))

In [ ]:
# 検証データを使用して予測精度を計算する
pred_val_y = model.predict(val_x_scaled)
r2_score(val_y_scaled, pred_val_y)

## ニューラルネットワークモデルを改良する


In [ ]:
tf.random.set_seed(0)
model = Sequential()
model.add(Dense(128, input_shape=(train_x.shape[1],))) # ニューロン数を3から128に変更
model.add(Activation('relu'))
model.add(Dense(128)) # ニューロン数を3から128に変更
model.add(Activation('relu'))
model.add(Dense(1))
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

In [ ]:
history = model.fit(train_x_scaled, train_y_scaled, batch_size=128, epochs=700, validation_data = (val_x_scaled, val_y_scaled), verbose=1)

In [ ]:
df_history = pd.DataFrame(history.history)
df_history.plot(figsize=(10, 6))

In [ ]:
# 検証データを使用して予測精度を計算する
pred_val_y = model.predict(val_x_scaled)
r2_score(val_y_scaled, pred_val_y)

### 学習不足と過学習
ニューロンを増やした学習では、最初は順調に誤差が小さくなっていますが、検証データに対しては100エポックを少し過ぎたあたりから、改善するどころかどんどん誤差が大きくなっています。

しかし、この付近ではニューロンが少ないモデルよりも検証データへの誤差が小さくなっています。

最初のニューロンの少ないモデルは、データに対して十分に適合できておらず、まだ改善の余地を残している適合不足の状態です。対して、ニューロンを増やしたモデルでは、訓練データに適合しすぎて過剰適合の状態です。

ニューラルネットワークの改良では、適合不足でもなく過剰適合でもない丁度よい状態を見つけることが必要になります。

また、ニューラルネットワークは他のモデルに比べて学習に非常に多くの時間がかかります。そこで、学習を効率的に進めることも合わせて考慮する必要があります。




## 早期終了（Early Stopping）を導入する
過学習が発生する一つの原因として、程よく訓練データに適合した状態を通り過ぎ、訓練データに過剰適合するまで学習を継続してしまったことがあります。

そこで、検証データに対する誤差が大きくなる前に、早期終了(Early Stopping)を行う設定を追加します。

早期終了は、学習時にfitメソッドの引数として、各エポック実行後に呼び出される、コールバック関数を設定することで実装できます。

In [ ]:
# モデルには先ほど学習したパラメータがすでに設定されているので、
# 学習状態をリセットするために再度モデルを定義します。モデルの内容は先ほどと変わりありません。
tf.random.set_seed(0)
model = Sequential()
model.add(Dense(128, input_shape=(train_x.shape[1],)))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(1))
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

In [ ]:
# EarlyStoppingの設定。patienceに指定した回数だけ連続で検証データの誤差が増えた場合に早期終了
es = EarlyStopping(monitor='val_loss',
                       patience=10,
                       verbose=1)

In [ ]:
# fitの引数にesを設定
history = model.fit(train_x_scaled, train_y_scaled, batch_size=128, epochs=700, validation_data = (val_x_scaled, val_y_scaled), verbose=1, callbacks=[es])

In [ ]:
df_history = pd.DataFrame(history.history)
df_history.plot(figsize=(10, 6))

In [ ]:
# 検証データを使用して予測精度を計算する
pred_val_y = model.predict(val_x_scaled)
r2_score(val_y_scaled, pred_val_y)

早期終了を入れると、学習にかかる時間も短くて済み、精度も向上しました。<br>

## 追加の最適化を導入する
ニューラルネットワークのさらなる最適化を行い精度向上を図ります。
今回は過学習を緩和する代表的な手法であるドロップアウトを適用してみます。

In [ ]:
tf.random.set_seed(0)
model = Sequential()
# 中間層1
model.add(Dense(128, input_shape=(train_x.shape[1],)))
model.add(Activation('relu')) 
model.add(Dropout(0.2)) # ドロップアウトの追加
# 中間層2
model.add(Dense(128))
model.add(Activation('relu')) 
model.add(Dropout(0.2)) # ドロップアウトの追加
# 出力層
model.add(Dense(1))
optimizer = optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse')
model.summary()

In [ ]:
history = model.fit(train_x_scaled, train_y_scaled, batch_size=128, epochs=700, validation_data = (val_x_scaled, val_y_scaled), verbose=1, callbacks=[es]) # EarlyStoppingを設定

In [ ]:
df_history = pd.DataFrame(history.history)
df_history.plot(figsize=(10, 6))

In [ ]:
# 検証データを使用して予測精度を計算する
pred_val_y = model.predict(val_x_scaled)
r2_score(val_y_scaled, pred_val_y)

ドロップアウトは代表的な最適化手法ですが、扱うデータにより必ず精度が向上するわけではありません。各層のノード数や層数、学習率、ミニバッチのサイズ、などを調整することで、さらに精度が向上する可能性があります。

このノートブックは以上です。